## Helpful Links
- Create New [GMail Account](https://accounts.google.com/signup)
- Turn [Allow less secure apps to ON](https://myaccount.google.com/lesssecureapps)
- [Text to HTML](https://wordtohtml.net/)

In [ ]:
import config
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [ ]:
# Test Data
user_name = 'Nick'
member_since = 2012

## Sending Email through Gmail

In [ ]:
sender_email = "SENDER_EMAIL"
receiver_email = "RECEIVER_EMAIL"
password = config.gmail_password

message = MIMEMultipart("alternative")
message["Subject"] = "Spotify Test Email"
message["From"] = sender_email
message["To"] = receiver_email

# Create the plain-text and HTML version of your message
html = f"""<p style="line-height: 1;">Hello {user_name},</p>
<p style="line-height: 1;"><br></p>
<p style="line-height: 1;">Thank you for listing to Spotify!</p>
<p style="line-height: 1;">You have been a member since {member_since}.&nbsp;</p>
<p style="line-height: 1;"><br></p>
<p style="line-height: 1;">Keep listing,</p>
<p style="line-height: 1;">Automated Email by Nick Drake</p>
"""

# Turn these into plain/html MIMEText objects
part2 = MIMEText(html, "html")

# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
# message.attach(part1)
# message.attach(part2)

# Create secure connection with server and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )